# Letter Case Permutations
[LeetCode #784](https://leetcode.com/problems/letter-case-permutation/)

Given a string, transform all letters to upper and lower case and return all permutations that can be formed this way.

* Example 1.
    - `str = "a12b"`
    - `expected_output = ["A12D", "A12b", "a12B", "a12b"]`
* Example 2.
    - `str = "abc"`
    - `expected_output = ['ABC', 'ABc', 'AbC', 'Abc', 'aBC', 'aBc', 'abC', 'abc']`

### Time Complexity Intuition
1. This is a n-choose-k problem. If the size of `n=4`, and the choice is to upper-case a letter, than how many total sets there are is the sum of:
    1. 4-C-0 (no letter's chosen to be uppercase) = 1
    2. 4-C-1 (1 letter chosen to be uppercased) =  4
    3. 4-C-2 (1 letter chosen to be uppercased) =  6
    4. 4-C-3 (1 letter chosen to be uppercased) =  4
    5. 4-C-4 (1 letter chosen to be uppercased) =  1
    6. The sum is **16** sets.
2. This analysis is how we can help ourselves explain Time Complexity of the solution. The strategy could be:
    1. Draw Pascals triangle at depth of N, and width of Choice.
    2. Explain how this diagram outlines a State-Space-Tree, that would mirror the Time Complexity of the recursion, which justify that the Worst Case time is going to be 2^n.
    3. You could further explain that 2^n is intuitively thought of as work doubling on every new level in the tree.
3. At each level in the tree, we'll make 2 choices: make the letter upper-case, or make the letter lower-case.
    1. If the letter is NOT a letter (it's a number), then we **skip** the work for that node, but continue to call recursively.  This is an important characteristic of this problem and potentially future problems like it - we do no work: simply call the next function once. Once the return comes back, we cleanup the work done by the function we called.
4. Whenever we've arrived at a length of 4, we'll take inventory of our proposed solution and insert it into our results.

### Designing the Solution
How to think about the solution

#### A. Strategy
0. Given that it's a nCk problem, we know we can find the result using a Brute-Force approach + State-Spaced-Tree.
1. Given 0, we should first understand the required depth of our tree - it will almost certainly be, size N.

In [ ]:
if n >= len(str):
    results.append(''.join(slate))
    return results

2. Next we should understand how many branches a single node in our tree will have. This will depend on the # of choices we're being offered in the problem description.
    - > ...transform all letters to upper and lower case
    - This clearly indicates we'll have 2-choices: Transform to UpperCase or Not.
3. Next we should observe if our solution will have a series of fixed sizes, or the same size? Why? Because the answer to this question will tell us if we're collecting results at the leaf nodes, or at some mid-level nodes.
    - > return all permutations that can be formed this way.
    - _Permutations_ initially seems to indicate the results will have dynamic sizes, but when we re-read the earlier part ...
    - > Given a string, transform all letters
    - We realize that a string is a fixed length, and we're simply to change individual letters. This means our answers will have a fixed size, which means we'll be collecting results at the leaf-nodes.

#### B. Tactics
1. Using a _slate_ we can work our way down to Pascals Triangle Depth, all the while, appending a part of the solution along the way.  This tactic works for solutions of a fixed size, or of variable sizes.  If the solution has variable sizes - then we can assume results will be collected at different levels in the tree. If the sizes are all fixed, then we can be sure the result will be collected at the level corresponding to the length of the result (usually the leaf node).
2. Let's disect the meat of this algorithm starting with the outtermost layer.

In [ ]:
char = str[n]                                           # TARGET: selecting a job
if char.isalpha():
    # WORK to be DONE for choosing 1 of 2, then 2 of 2 choices.
else:
    slate[n] = char
    # NO WORK to be DONE, so skipping WORK
return results

The above code, demonstrates how there's a separation of concerns between which nodes do some WORK and which nodes simply don't do any work, but don't want to break the chain of events.  This type of structure indicates that a **constraint** exists in the problem.
- Example:
    - > Do some work, but only when this conditions is True.
We might be tempted to think; this means we should return from calling any more recursive calls: but that would be VERY WRONG. We need to continue calling recursively, we simply don't do any WORK, or make any choices.

A Good Analogy:  You're running a relay race, and a series of runners before you ran 400 meters, than passed off the baton to you, once you have the batton, you don't run at all, and immediately pass the batton off to the next runner.

Another Analogy:  You and some friends are walking different paths and they all find fork's in their paths, but you don't find any.

In [ ]:
char = str[n]                                           # TARGET: selecting a job
if char.isalpha():
    slate[n] = char.upper()                             # WORK: Building up our next result
                                                        # CHOOSING 1st of 2 options
    letter_case_permutation(str, n+1, slate, results)   # handing the WORK over to the next WORKER.

    slate[n] = char.lower()                             # BACKTRACKING:
                                                        # Building up our next result
                                                        # CHOOSING 2nd of 2 options

    letter_case_permutation(str, n+1, slate, results)   # handing the WORK over to the next WORKER
else:
    slate[n] = char
    letter_case_permutation(str, n+1, slate, results)
return results

3. Zooming into the finer details, we can see what DOING WORK looks like; A node is responsible for picking up a job and then immediately casting it to uppercase if it's a letter. This is making the first of 2 choices.  This node sits in the call stack and waits for his turn to do some more work after he calls another node to work on top of his work.  Once the first call is finished, we make another transformation and call the other nodes again to work on top of our second choice.  Finally when both of those tasks are complete, we return up the call stack, signaling our job is done.

2. We can use _Backtracking_ to undo our previous call's work done to the _slate_, to then modify our next possible answer.

In [ ]:
letter_case_permutation(str, n+1, slate, results)   # handing the WORK over to the next WORKER.

slate[n] = char.lower()                             # BACKTRACKING:
                                                    # Building up our next result
                                                    # CHOOSING 2nd of 2 options
letter_case_permutation(str, n+1, slate, results)   # handing the WORK over to the next WORKER

3. It should be noted that these 2 lines produce the *Backtracking* effect in this algorithm - we're overwriting our previous work with new work then making another call.
4. The algorithm signature will have a recursive call to pass off the work responsbility to another node.
5. The changes we make after this call, and before the next call will represent the different choice we're allowed to make for any given node.
6. Once that node is finished, and all the sub-ordinate nodes' work has been "collected" in the form of a result, we can "undo" the staging of that result.
7. In almost every case, there will be a constraint, or series of constraints defined in the problem. These constraints are how we have so many unique situations to solve. These constraints will also be **vital clues** to helping us justify our runtime complexity and determining the _Bounding Function_ of our algorithm (edge case).

In [41]:
def letter_case_permutation(str, n=0, slate=None, results=[]):
    if slate is None:
        slate = [None] * len(str)
    if n >= len(str):
        results.append(''.join(slate))
        return results
    char = str[n]
    if char.isalpha():
        slate[n] = char.upper()
        letter_case_permutation(str, n+1, slate, results)
        slate[n] = char.lower()
        letter_case_permutation(str, n+1, slate, results)
    else:
        slate[n] = char
        letter_case_permutation(str, n+1, slate, results)
    return results

print("Answer: ", letter_case_permutation('abc'))

Answer:  ['ABC', 'ABc', 'AbC', 'Abc', 'aBC', 'aBc', 'abC', 'abc']
